In [400]:
import requests
import pandas as pd
import folium
from folium.plugins import HeatMap


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 500)

In [5]:
start_time = 'now-180days'
min_magnitude = 3
latitude = 39.1458
longitude = 34.1614
max_radius_km = 1500

In [317]:
url = requests.get(f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_time}&minmagnitude={min_magnitude}&latitude={latitude}&longitude={longitude}&maxradiuskm={max_radius_km}')
dataset = url.json()

In [7]:
url

<Response [200]>

In [318]:
features = dataset['features']

In [319]:
places = []
mags = []
times = []
lats = []
lons = []
weights = []

for feature in features:
    places.append(feature['properties']['place'])
    mags.append(feature['properties']['mag'])
    times.append(pd.to_datetime(feature['properties']['time'], unit='ms').strftime('%y/%m/%d %H:%M:%S'))
    lats.append(feature['geometry']['coordinates'][1])
    lons.append(feature['geometry']['coordinates'][0])
    weights.append(feature['properties']['mag'])

In [320]:
dataset_earthquake = pd.DataFrame({
                            'Place': places,
                            'Magnitude': mags,
                            'Time': times,
                            'Latitude': lats,
                            'Longitude': lons,
                            'Weight': weights
                            })

In [321]:
df = dataset_earthquake.copy()

In [322]:
df

,Place,Magnitude,Time,Latitude,Longitude,Weight
0,Iran-Iraq border region,4.8,23/03/25 19:13:32,32.6147,47.6789,4.8
1,"12 km SE of Saimbeyli, Turkey",4.3,23/03/25 06:33:40,37.9055,36.1849,4.3
2,"19 km WSW of Ekinözü, Turkey",4.4,23/03/25 04:39:34,37.9894,36.9839,4.4
3,"7 km ENE of Nurhak, Turkey",4.4,23/03/24 13:10:34,37.9773,37.5260,4.4
4,Azerbaijan,4.1,23/03/24 07:10:02,40.8647,48.2451,4.1
...,...,...,...,...,...,...
681,eastern Turkey,4.4,22/09/27 16:16:56,40.7747,42.9258,4.4
682,"19 km N of Datça, Turkey",4.0,22/09/27 15:46:32,36.9112,27.6954,4.0
683,"10 km WNW of Susuz, Turkey",4.0,22/09/27 14:43:39,40.8132,43.0097,4.0
684,"19 km W of Susuz, Turkey",4.3,22/09/27 14:24:40,40.7497,42.9007,4.3


In [323]:
df.isnull().sum()

Place        33
Magnitude     0
Time          0
Latitude      0
Longitude     0
Weight        0
dtype: int64

In [324]:
df['Place'].fillna("NoInformation", inplace= True)

In [326]:
turkey_coord = [39, 35]
turkey_map_normal = folium.Map(location=turkey_coord, zoom_start=5.5)
HeatMap(data=df[['Latitude', 'Longitude', 'Weight']], radius=15).add_to(turkey_map_normal)

for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude']/2,
        color='red',
        fill_color='red').add_to(turkey_map_normal)

turkey_map_normal

In [414]:
turkey_coord = [39, 35]
turkey_map_dark = folium.Map(location=turkey_coord, zoom_start=5.5,  tiles='cartodbdark_matter')
HeatMap(data=df[['Latitude', 'Longitude', 'Weight']], radius=15).add_to(turkey_map_dark)

for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude']/2,
        color='red',
        fill_color='red').add_to(turkey_map_dark)

turkey_map_dark

## Data Manipulation:

In [328]:
df

,Place,Magnitude,Time,Latitude,Longitude,Weight
0,Iran-Iraq border region,4.8,23/03/25 19:13:32,32.6147,47.6789,4.8
1,"12 km SE of Saimbeyli, Turkey",4.3,23/03/25 06:33:40,37.9055,36.1849,4.3
2,"19 km WSW of Ekinözü, Turkey",4.4,23/03/25 04:39:34,37.9894,36.9839,4.4
3,"7 km ENE of Nurhak, Turkey",4.4,23/03/24 13:10:34,37.9773,37.5260,4.4
4,Azerbaijan,4.1,23/03/24 07:10:02,40.8647,48.2451,4.1
...,...,...,...,...,...,...
681,eastern Turkey,4.4,22/09/27 16:16:56,40.7747,42.9258,4.4
682,"19 km N of Datça, Turkey",4.0,22/09/27 15:46:32,36.9112,27.6954,4.0
683,"10 km WNW of Susuz, Turkey",4.0,22/09/27 14:43:39,40.8132,43.0097,4.0
684,"19 km W of Susuz, Turkey",4.3,22/09/27 14:24:40,40.7497,42.9007,4.3


In [365]:
for place in list(df['Place'].unique()):
    if "," in place and "border" not in place:
        if "Kahramanmaras" in place:
            df.loc[(df['Place'] == place), 'Country'] = 'Turkey'
        else:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(',')[-1])
    elif "," not in place and "border" not in place and " " in place and len(list(place.split(' ')))>1 and len(list(place.split(' ')))<3:
        if "Iran" in place:
            df.loc[(df['Place'] == place), 'Country'] = "Iran"
        elif "Turkey" in place:
            df.loc[(df['Place'] == place), 'Country'] = "Turkey"
        elif "Greece" in place:
            df.loc[(df['Place'] == place), 'Country'] = "Greece"
    elif "," not in place and "border" not in place and " " in place :
        if "Kahramanmaras" in place:
             df.loc[(df['Place'] == place), 'Country'] = 'Turkey'
        elif "region" in place:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(' ')[0])
        elif "Sea" in place:
            df.loc[(df['Place'] == place), 'Country'] = place
        elif "coast" in place:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(' ')[-1])
        else:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(' ')[1])
    elif "border" in place:
        df.loc[(df['Place'] == place), 'Country'] = place
    
    else:
        df.loc[(df['Place'] == place), 'Country'] = place
        
        
df.loc[((df['Country'].isnull()) & (df['Place'].str.contains('Cyprus'))), 'Country' ] = 'Cyprus'
df.loc[((df['Country'].isnull()) & (df['Place'].str.contains('Sea'))), 'Country' ] = df['Place']        

In [371]:
for place in list(df['Place'].unique()):
    if "," in place:
        df.loc[(df['Place'] == place), 'City'] = df['Place'].apply(lambda x: x.split(',')[0].split(' ')[-1])
        

In [373]:
df['City'].fillna('NO CITY INFORMATION' ,inplace = True)

In [375]:
df['Date'] = df['Time'].apply(lambda x: x.split(" ")[0])
df['TimeZone'] = df['Time'].apply(lambda x: x.split(" ")[1])

In [376]:
df

,Place,Magnitude,Time,Latitude,Longitude,Weight,Country,City,Date,TimeZone
0,Iran-Iraq border region,4.8,23/03/25 19:13:32,32.6147,47.6789,4.8,Iran-Iraq border region,NO CITY INFORMATION,23/03/25,19:13:32
1,"12 km SE of Saimbeyli, Turkey",4.3,23/03/25 06:33:40,37.9055,36.1849,4.3,Turkey,Saimbeyli,23/03/25,06:33:40
2,"19 km WSW of Ekinözü, Turkey",4.4,23/03/25 04:39:34,37.9894,36.9839,4.4,Turkey,Ekinözü,23/03/25,04:39:34
3,"7 km ENE of Nurhak, Turkey",4.4,23/03/24 13:10:34,37.9773,37.5260,4.4,Turkey,Nurhak,23/03/24,13:10:34
4,Azerbaijan,4.1,23/03/24 07:10:02,40.8647,48.2451,4.1,Azerbaijan,NO CITY INFORMATION,23/03/24,07:10:02
...,...,...,...,...,...,...,...,...,...,...
681,eastern Turkey,4.4,22/09/27 16:16:56,40.7747,42.9258,4.4,Turkey,NO CITY INFORMATION,22/09/27,16:16:56
682,"19 km N of Datça, Turkey",4.0,22/09/27 15:46:32,36.9112,27.6954,4.0,Turkey,Datça,22/09/27,15:46:32
683,"10 km WNW of Susuz, Turkey",4.0,22/09/27 14:43:39,40.8132,43.0097,4.0,Turkey,Susuz,22/09/27,14:43:39
684,"19 km W of Susuz, Turkey",4.3,22/09/27 14:24:40,40.7497,42.9007,4.3,Turkey,Susuz,22/09/27,14:24:40


In [378]:
df['Year\Month'] = df['Date'].apply(lambda x : str(x.split("/")[0]) + "/" +  str(x.split("/")[1]))

In [383]:
for country in df['Country']:
    df.loc[(df['Place'].str.contains(f'{country}')), 'Country'] = country 

## Data Analysis :

In [386]:
df

,Place,Magnitude,Time,Latitude,Longitude,Weight,Country,City,Date,TimeZone,Year\Month
0,Iran-Iraq border region,4.8,23/03/25 19:13:32,32.6147,47.6789,4.8,Iran,NO CITY INFORMATION,23/03/25,19:13:32,23/03
1,"12 km SE of Saimbeyli, Turkey",4.3,23/03/25 06:33:40,37.9055,36.1849,4.3,Turkey,Saimbeyli,23/03/25,06:33:40,23/03
2,"19 km WSW of Ekinözü, Turkey",4.4,23/03/25 04:39:34,37.9894,36.9839,4.4,Turkey,Ekinözü,23/03/25,04:39:34,23/03
3,"7 km ENE of Nurhak, Turkey",4.4,23/03/24 13:10:34,37.9773,37.5260,4.4,Turkey,Nurhak,23/03/24,13:10:34,23/03
4,Azerbaijan,4.1,23/03/24 07:10:02,40.8647,48.2451,4.1,Azerbaijan,NO CITY INFORMATION,23/03/24,07:10:02,23/03
...,...,...,...,...,...,...,...,...,...,...,...
681,eastern Turkey,4.4,22/09/27 16:16:56,40.7747,42.9258,4.4,Turkey,NO CITY INFORMATION,22/09/27,16:16:56,22/09
682,"19 km N of Datça, Turkey",4.0,22/09/27 15:46:32,36.9112,27.6954,4.0,Turkey,Datça,22/09/27,15:46:32,22/09
683,"10 km WNW of Susuz, Turkey",4.0,22/09/27 14:43:39,40.8132,43.0097,4.0,Turkey,Susuz,22/09/27,14:43:39,22/09
684,"19 km W of Susuz, Turkey",4.3,22/09/27 14:24:40,40.7497,42.9007,4.3,Turkey,Susuz,22/09/27,14:24:40,22/09


## How many earthquakes have occurred in which country in the last 6 months?

In [389]:
df.groupby('Country').agg({"Weight": "count"}).sort_values(by = "Weight", ascending = False)[:10]

,Weight
Country,
Turkey,508
Greece,61
NoInformation,33
Iran,22
Syria,14
Romania,9
Albania,8
Cyprus,6
Russia,3


## What is the order of the Magnitudes of the earthquakes that will occur?

In [394]:
mean_std = df.groupby('Country').agg({"Magnitude": "mean",
                          "Weight" : "std"}).sort_values(by = "Magnitude", ascending = False)[:10]

mean_std.columns = ['Magnitude_mean', 'Magnitude_std']
mean_std

,Magnitude_mean,Magnitude_std
Country,,
Iraq,4.850000,0.353553
eastern Mediterranean Sea,4.800000,NaN
Romania,4.688889,0.491031
Russia,4.633333,0.776745
Iran,4.622727,0.545922
Aegean Sea,4.550000,0.070711
NoInformation,4.509091,0.309600
Turkey,4.442913,0.426315
Greece,4.439344,0.402608


## We know that earthquakes have occurred in the last 6 months. So, how many earthquakes were there in which month and what are their magnitudes?

In [396]:
df

,Place,Magnitude,Time,Latitude,Longitude,Weight,Country,City,Date,TimeZone,Year\Month
0,Iran-Iraq border region,4.8,23/03/25 19:13:32,32.6147,47.6789,4.8,Iran,NO CITY INFORMATION,23/03/25,19:13:32,23/03
1,"12 km SE of Saimbeyli, Turkey",4.3,23/03/25 06:33:40,37.9055,36.1849,4.3,Turkey,Saimbeyli,23/03/25,06:33:40,23/03
2,"19 km WSW of Ekinözü, Turkey",4.4,23/03/25 04:39:34,37.9894,36.9839,4.4,Turkey,Ekinözü,23/03/25,04:39:34,23/03
3,"7 km ENE of Nurhak, Turkey",4.4,23/03/24 13:10:34,37.9773,37.5260,4.4,Turkey,Nurhak,23/03/24,13:10:34,23/03
4,Azerbaijan,4.1,23/03/24 07:10:02,40.8647,48.2451,4.1,Azerbaijan,NO CITY INFORMATION,23/03/24,07:10:02,23/03
...,...,...,...,...,...,...,...,...,...,...,...
681,eastern Turkey,4.4,22/09/27 16:16:56,40.7747,42.9258,4.4,Turkey,NO CITY INFORMATION,22/09/27,16:16:56,22/09
682,"19 km N of Datça, Turkey",4.0,22/09/27 15:46:32,36.9112,27.6954,4.0,Turkey,Datça,22/09/27,15:46:32,22/09
683,"10 km WNW of Susuz, Turkey",4.0,22/09/27 14:43:39,40.8132,43.0097,4.0,Turkey,Susuz,22/09/27,14:43:39,22/09
684,"19 km W of Susuz, Turkey",4.3,22/09/27 14:24:40,40.7497,42.9007,4.3,Turkey,Susuz,22/09/27,14:24:40,22/09


In [407]:
    df.groupby(['Year\Month','Country']).agg({"Magnitude": "count"}
                                          ).reset_index().sort_values(by = "Magnitude", ascending = False)[:15]
                                                                                                           

,Year\Month,Country,Magnitude
52,23/02,Turkey,426
63,23/03,Turkey,44
51,23/02,NoInformation,22
36,23/01,Greece,17
11,22/11,Greece,13
38,23/01,Iran,12
46,23/02,Syria,10
49,23/02,Greece,9
6,22/10,Turkey,9
22,22/12,Greece,8


## I made the distribution according to the countries. How many earthquakes occurred in which city of which country?

In [408]:
   df.groupby(['Year\Month','Country', 'City']).agg({"Magnitude": "count"}
                                          ).reset_index().sort_values(by = "Magnitude", ascending = False)[:15]

,Year\Month,Country,City,Magnitude
172,23/02,Turkey,NO CITY INFORMATION,77
151,23/02,Turkey,Göksun,55
174,23/02,Turkey,Nurhak,26
179,23/02,Turkey,Sincik,22
129,23/02,NoInformation,NO CITY INFORMATION,22
189,23/02,Turkey,Çelikhan,19
173,23/02,Turkey,Nurda??,19
176,23/02,Turkey,Pazarc?k,18
144,23/02,Turkey,Do?an?ehir,17
186,23/02,Turkey,Ye?ilyurt,14


In [412]:
table_ea = df[df['Country'] != "NoInformation"].groupby(['Year\Month','Country', 'City']).agg({"Magnitude": "count",
                                                      "Weight" : "mean"}).sort_values(by = "Magnitude", ascending = False)

table_ea.columns = ['Count', 'Magnitude']
table_ea

Count  Magnitude
Year\Month Country                         City                                 
23/02      Turkey                          NO CITY INFORMATION     77      4.413
                                           Göksun                  55      4.445
                                           Nurhak                  26      4.469
                                           Sincik                  22      4.427
                                           Çelikhan                19      4.389
                                           Nurda??                 19      4.521
                                           Pazarc?k                18      4.533
                                           Do?an?ehir              17      4.588
                                           Ye?ilyurt               14      4.571
                                           Ekinözü                 14      4.486
                                           Uzunba?                 12      4.642
23/01      Iran                            Khowy                   12      4.642
23/02      Turkey                          Tut                     11      4.464
                                           Kahramanmara?           10      4.250
23/03      Turkey                          Göksun                   9      4.500
23/02      Turkey                          Karafak?l?               8      4.475
            Syria                          Latakia                  7      4.329
           Turkey                          Gölba??                  7      4.500
                                           Ça?layancerit            7      4.329
                                           Af?in                    6      4.517
                                           Palu                     6      4.467
                                           K?r?khan                 6      4.300
                                           Hassa                    6      4.433
23/03      Turkey                          NO CITY INFORMATION      5      4.240
                                           Sincik                   5      4.360
23/02      Turkey                          Ad?yaman                 4      4.300
                                           Darende                  4      4.350
22/09      Turkey                          Susuz                    3      4.533
23/03       Albania                        Ersekë                   3      4.433
23/01      Greece                          Crete                    3      4.267
23/02      Turkey                          Yaylada??                3      4.267
23/03      Turkey                          Ekinözü                  3      4.367
23/02      Turkey                          Bahçe                    3      4.267
                                           Uzunkavak                3      4.367
                                           Besni                    3      4.500
            Syria                          NO CITY INFORMATION      2      4.750
            Georgia                        Ozurgeti                 2      4.200
           Turkey                          Büyükçat                 2      4.700
           Greece                          Kýthira                  2      4.500
           Turkey                          Dalyan                   2      4.600
23/01      Turkey                          NO CITY INFORMATION      2      4.400
23/02      Turkey                          Saimbeyli                2      4.450
                                           Do?anyol                 2      4.350
23/01      Greece                          Koróni                   2      4.400
23/02      Turkey                          Hani                     2      4.300
                                           Dörtyol                  2      4.250
                                           Musabeyli                2      4.150
                                           Malatya                  2      4.250
23/01       Al